In [27]:
import pandas as pd

#upload the file
predicted_value = pd.read_excel("clean data before python.xlsx",sheet_name="predicted_value",header=None, skiprows=12)
metric = pd.read_excel("clean data before python.xlsx",sheet_name="predicted_value",header=None, nrows=11).T
auditInfo = pd.read_excel("clean data before python.xlsx",sheet_name="Audit Information", skiprows=3)
metric.columns = metric.iloc[0]
metric = metric.drop(0)

In [2]:
#transform from pivot to raw data
dfAll = pd.DataFrame()
for i in range(predicted_value.shape[0]) :
    tempSeries = predicted_value.iloc[i]
    theID = tempSeries[0]
    dfTemp = metric
    dfTemp["New_Audit_id"] = theID
    dfTemp["predictedValue"] = tempSeries[1:]
    dfAll = pd.concat([dfAll,dfTemp])
#make new ID
dfAll["predicted_id"] = [ a+ "_" + str(b) for a,b in zip(dfAll["New_Audit_id"],dfAll["Remarks"])]
#rearrange data
dfAll = dfAll[["predicted_id",'New_Audit_id', 'Metric:', 'Case:', 'Depth:', 'Energy:',
      'Energy_FFF :', 'Gantry:', 'Mode:', 'Point type',
      'predictedValue']] 
dfAll = dfAll.drop_duplicates()
dfFilteres = dfAll[dfAll["predictedValue"].isna() == False]
combinedAll = auditInfo.merge(dfFilteres,on="New_Audit_id")

In [3]:
combinedAll = combinedAll[['predicted_id','New_Audit_id', 'Audit_ID', 'Facility_ID',
       'Status', 'Auditor1', 'Auditor2', 'Auditor3', 'Au date', 'Manuf',
       'Model', 'TPS', 'Algorithm', 'Matched', 'k_Q Fac.', 'k_Q ACDS', 'Array',
       'S.N.', 'audit', '6_k_user', '10_k_user', '15_k_user', '18_k_user',
       '6FFF_k_user', '10FFF_k_user', '6_Daily_Output', '10_Daily_Output',
       '15_Daily_Output', '18_Daily_Output', '6FFF_Daily_Output',
       '10FFF_Daily_Output', '6__TPR_20,10', '10__TPR_20,10', '15__TPR_20,10',
       '18__TPR_20,10', '6FFF__TPR_20,10', '10FFF__TPR_20,10', 'version',
       'corr.', 'Alg.', 'Rev.',
       'Metric:', 'Case:', 'Depth:', 'Energy:', 'Energy_FFF :', 'Gantry:',
       'Mode:', 'Point type', 'predictedValue']]

In [4]:
# delete unused case 4,5,11,12
combinedAll = combinedAll[~combinedAll['Case:'].isin([4,5,11,12])].reset_index(drop=True)

In [25]:
combinedAll[combinedAll["New_Audit_id"] == "928438_2"]

,predicted_id,New_Audit_id,Audit_ID,Facility_ID,Status,Auditor1,Auditor2,Auditor3,Au date,Manuf,...,Rev.,Metric:,Case:,Depth:,Energy:,Energy_FFF :,Gantry:,Mode:,Point type,predictedValue


In [24]:
#get list 
listDupAuditId =  list(set([int(auditId[0]) for auditId in combinedAll.New_Audit_id.str.split("_") if int(auditId[1]) > 1]))
combinedAll[(combinedAll['Audit_ID'].isin(listDupAuditId))].to_csv("result data of duplicate ID.csv",index=False,sep = "\t",na_rep="NA")
combinedAll[(combinedAll['Audit_ID'].isin(listDupAuditId))].to_excel("result data of duplicate ID.xlsx",index = False,na_rep="NA")

In [7]:
PrescriptionPointData = combinedAll[combinedAll['Point type'] == "Prescription Point"].copy()

In [8]:
PrescriptionPointData.pivot_table(index=['New_Audit_id', 'Energy:'], columns='Case:', values='predictedValue', aggfunc='count')

Case:                  2    3    6    7    8    9    10
New_Audit_id Energy:                                   
103854_1     6        1.0  2.0  2.0  1.0  2.0  4.0  2.0
             10       NaN  1.0  1.0  1.0  1.0  2.0  2.0
106135_1     6        1.0  1.0  1.0  1.0  1.0  2.0  2.0
             10       NaN  1.0  1.0  1.0  1.0  NaN  2.0
106836_1     6        1.0  1.0  1.0  1.0  NaN  2.0  2.0
...                   ...  ...  ...  ...  ...  ...  ...
996247_2     18       1.0  2.0  1.0  1.0  1.0  2.0  2.0
996724_1     6        1.0  1.0  1.0  NaN  1.0  2.0  NaN
             15       NaN  1.0  NaN  NaN  NaN  NaN  NaN
997736_1     6        1.0  1.0  1.0  1.0  1.0  2.0  2.0
             10       NaN  1.0  1.0  1.0  1.0  NaN  2.0

[641 rows x 7 columns]

In [24]:
combinedAll.to_excel("result clean data.xlsx",index = False,na_rep="NA")
combinedAll.to_csv("result clean data.csv",index=False,sep = "\t",na_rep="NA")